# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: distillbert-base-uncased
* Evaluation approach: classification
* Fine-tuning dataset:  yelp reviews

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset
import torch

from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM, AutoTokenizer
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType,PeftModel
import gc


In [46]:
# load a dataset yelp review 
ds = load_dataset("fancyzhx/yelp_polarity", split="train[42%:50%]").train_test_split(test_size=0.2, shuffle=True, seed=23)

In [47]:
# load tokenizer

model_name = 'distilbert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)



In [48]:
# apply tokenizer to dataset
splits = ["train", "test"]

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = ds[split].map(tokenize_function, batched=True)


Map:   0%|          | 0/8960 [00:00<?, ? examples/s]

In [49]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Training and Evaluating

In [8]:
# load model from huggingface

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=2,
                                                           id2label={0: "bad", 1: "good"},
                                                           label2id={"bad": 0, "good": 1}).to("cuda")


for param in model.base_model.parameters():
    param.requires_grad = False

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/classifier_head",
        # Set the learning rate
        learning_rate = 0.001,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size = 16,
        per_device_eval_batch_size = 8,
        # Evaluate and save the model after each epoch
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        num_train_epochs=1,
        weight_decay=0.001,
        load_best_model_at_end=True,
        fp16=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


# Training the classifier head

Instruction mentions to just load the model and evaluate it, however, the pretrained model needs a classifier on top which is not trained. So, to make the evaluation meaningful, I train the classifier head first before evaluating the model. 

In [10]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.275000,0.240385,0.900335


TrainOutput(global_step=2240, training_loss=0.290946900844574, metrics={'train_runtime': 321.6664, 'train_samples_per_second': 111.42, 'train_steps_per_second': 6.964, 'total_flos': 4747631567831040.0, 'train_loss': 0.290946900844574, 'epoch': 1.0})

In [11]:
# Save model
model_name = 'distillbert_' + 'classifier'
model.save_pretrained('./data/' + model_name)


In [12]:
trainer.evaluate()

{'eval_loss': 0.24038466811180115,
 'eval_accuracy': 0.9003348214285715,
 'eval_runtime': 58.5362,
 'eval_samples_per_second': 153.068,
 'eval_steps_per_second': 19.133,
 'epoch': 1.0}

### Load model from drive


In [13]:
model_name = 'distillbert_classifier'
model_path = '/workspace/data/' + model_name
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=2,
                                                           id2label={0: "bad", 1: "good"},
                                                           label2id={"bad": 0, "good": 1}).to("cuda")



In [14]:
# Test model 
df = pd.DataFrame(tokenized_dataset["test"])
df = df[["text", "label"]]
tokenized_text = tokenizer(list(df["text"]), padding="max_length", truncation=True, return_tensors='pt')

In [85]:
# Evaluate saved model
batch_size = 32
b = [0, batch_size]

len_samples = len(tokenized_text['input_ids'])

predicted_labels = []
model.eval()
while b[1] < len_samples:
    print(round(b[1]/len_samples, 3), end = '\r')
    with torch.no_grad():
        outputs = model(tokenized_text['input_ids'][b[0]:b[1]].to('cuda'))
        predicted_labels.append(outputs.logits.argmax(-1))
        b[0] += batch_size
        b[1] += batch_size

with torch.no_grad():
    outputs = model(tokenized_text['input_ids'][b[0]:b[1]].to('cuda'))
    predicted_labels.append(outputs.logits.argmax(-1))



In [16]:
# Evaluate model 
predictions = np.concatenate(([list(np.array(pp.cpu().numpy())) for pp in predicted_labels]))
accuracy = np.sum(predictions == list(df['label']))/len(df['label'])
print(f"accuracy {accuracy}")

accuracy 0.7338169642857143


In [17]:
# Check some of the texts that were incorrectly labeled 
df['predicted_label'] = predictions
incorrect_predict_reviews = df[df['label'] != predictions]
incorrect_predict_reviews.iloc[0]['text']


'Best coffee in town. You never would think a donut shop would be known for their coffee but Dunkin Donuts has excellent coffee. The donuts are slightly above average. But over all a good quick on the go breakfast.'

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [69]:
from peft import LoraConfig
from peft import get_peft_model

model_name = 'distillbert_classifier'
model_path = '/workspace/data/' + model_name

model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=2,
                                                           id2label={0: "bad", 1: "good"},
                                                           label2id={"bad": 0, "good": 1})


In [71]:
# Create a PEFT Model
lora_config = LoraConfig(target_modules=["q_lin", "k_lin","v_lin"], # Which layer to apply LoRA, usually only apply on MultiHead Attention Layer
                         task_type=TaskType.SEQ_CLS )

lora_model = get_peft_model(model, lora_config).to('cuda')
lora_model.print_trainable_parameters()

trainable params: 1,405,444 || all params: 67,768,324 || trainable%: 2.073895172617815


In [73]:
del(model)
torch.cuda.empty_cache()
gc.collect()

10704

In [74]:
# Train the PEFT Model
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/lora",
        learning_rate = 0.001,
        per_device_train_batch_size = 4,
        per_device_eval_batch_size = 4,
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        num_train_epochs=1,
        weight_decay=0.001,
        load_best_model_at_end=True,
        fp16=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.178900,0.171411,0.950335


Checkpoint destination directory ./data/lora/checkpoint-8960 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=8960, training_loss=0.22954959869384767, metrics={'train_runtime': 816.1692, 'train_samples_per_second': 43.912, 'train_steps_per_second': 10.978, 'total_flos': 4837177829621760.0, 'train_loss': 0.22954959869384767, 'epoch': 1.0})

In [75]:
trainer.evaluate()

{'eval_loss': 0.17141121625900269,
 'eval_accuracy': 0.9503348214285714,
 'eval_runtime': 73.5817,
 'eval_samples_per_second': 121.769,
 'eval_steps_per_second': 30.442,
 'epoch': 1.0}

In [76]:
# Save the PEFT model
model_name = 'distillbert_' + 'lora'
lora_model.save_pretrained('./data/' + model_name)


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [79]:
# Load the PEFT Model
model_name = './data/' +  'distillbert_' + "lora"

lora_model = AutoPeftModelForSequenceClassification.from_pretrained(model_name).to('cuda')


In [80]:
torch.cuda.empty_cache()
gc.collect()

29

In [81]:
# Test model 
df = pd.DataFrame(tokenized_dataset["test"])
df = df[["text", "label"]]
tokenized_text = tokenizer(list(df["text"]), padding="max_length", truncation=True, return_tensors='pt')

In [82]:
# Evaluate saved model
batch_size = 32
b = [0, batch_size]

len_samples = len(tokenized_text['input_ids'])

predicted_labels = []
lora_model.eval()
while b[1] < len_samples:
    print(round(b[1]/len_samples, 3), end = '\r')
    with torch.no_grad():
        outputs = lora_model(tokenized_text['input_ids'][b[0]:b[1]].to('cuda'))
        predicted_labels.append(outputs.logits.argmax(-1))
        b[0] += batch_size
        b[1] += batch_size

with torch.no_grad():
    outputs = lora_model(tokenized_text['input_ids'][b[0]:b[1]].to('cuda'))
    predicted_labels.append(outputs.logits.argmax(-1))


In [83]:
# Evaluate model 
predictions = np.concatenate(([list(np.array(pp.cpu().numpy())) for pp in predicted_labels]))
accuracy = np.sum(predictions == list(df['label']))/len(df['label'])
print(f"accuracy {accuracy}")

accuracy 0.8800223214285714


In [84]:
# Check some of the texts that were incorrectly labeled 
df['lora_predicted_label'] = predictions
incorrect_predict_reviews = df[df['label'] != predictions]
incorrect_predict_reviews.iloc[0]['text']


'Islands \\""fine\\"" burgers.... Should drop the \\""fine\\"" in their name. This is simply another version of Chilis or AppleBee\'s. Frozen burgers, frozen fries, frozen veg. - overall pretty blah :-('